In [15]:
import RAKE
import nltk
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stop = stopwords.words('russian')

In [5]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
m = MorphAnalyzer()

In [6]:
rake = RAKE.Rake(stop)

In [16]:
vectorizer = TfidfVectorizer()

In [8]:
from gensim.summarization import keywords as kw_gs

In [9]:
def get_text(filename):
    with open (filename, encoding = 'utf-8') as f:
        content = f.read()
    key_words, my_key_words, text = content.split('\n\n\n')
    return key_words.strip('\ufeff').split('\n'), my_key_words.split('\n'), text

In [10]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)


Тексты для моего мини-корпуса взяты с сайта НИУ ВШЭ (раздел "Новости"), где ключевые слова помечены как "темы", есть еще отдельная метка "рубрика" - ее содержание не включалось в список ключевых слов.

In [11]:
files = ('control.txt', 'vvp.txt', 'career_marathon.txt', 'social.txt')

In [71]:
for file in files:
    key_words, my_key_words, text = get_text(file)
    
    #rake
    kw_list = rake.run(normalize_text(text), maxWords=5, minFrequency=2)
    kw_rake = [x[0] for x in kw_list]
    
    #TextRank
    kw_gensim = kw_gs(normalize_text(text), pos_filter=[], scores=True)
    kw_gensim = [x[0] for x in kw_gensim if x[1]>0.15]
    
    #tf-idf
    corpus = sent_tokenize(text)
    X = vectorizer.fit_transform(corpus)
    scores = zip(vectorizer.get_feature_names(), np.asarray(X.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    tf_list = [x[0] for x in sorted_scores if x[1]>1]
    
    #precision, recall, f1
    print('Results for text', file)
    precision = len(set(kw_rake).intersection(set(my_key_words)))/len(my_key_words)
    recall = len(set(kw_rake).intersection(set(my_key_words)))/len(kw_rake)
    if precision != 0 and recall != 0:
        f1 = 2*(recall*precision)/(recall+precision)
    else:
        f1 = 0
    print('Precision', precision, '\n'
         'Recall', recall, '\n',
         'F1', f1, '\n')
    print ('REAL KEY WORDS\n', key_words)
    print('PREDICTED\n',
          'rake', kw_rake, '\n',
         'textrank', kw_gensim, '\n',
          'tf-idf', tf_list,'\n')

Results for text control.txt
Precision 0.4 
Recall 0.16666666666666666 
 F1 0.23529411764705882 

REAL KEY WORDS
 ['лектории', 'Университет, открытый городу', 'новое в ВШЭ', 'приглашение к участию']
PREDICTED
 rake ['акция « выходить решать', 'сбор участник', 'контрольный', 'физика', 'информатика', 'корпус', 'ауд', '»', '11', '10', '30', '00'] 
 textrank ['контрольныи', 'она'] 
 tf-idf ['на', '00', 'контрольной', '10', 'ауд'] 

Results for text vvp.txt
Precision 0.0 
Recall 0.0 
 F1 0 

REAL KEY WORDS
 ['цифра дня', 'исследования и аналитика', 'экспертиза', 'статистические данные']
PREDICTED
 rake ['8 миллиард рубль', 'это', '3', '5 %', '1', '1 %', '2 %', '0'] 
 textrank ['цифровои', 'миллиард', 'исиэз', 'год', 'триллион рубль', 'исследование', 'потратить', 'внутреннии затрата'] 
 tf-idf ['на', 'руб', 'млрд', 'трлн'] 

Results for text career_marathon.txt
Precision 0.25 
Recall 0.2 
 F1 0.22222222222222224 

REAL KEY WORDS
 ['выпускники', 'студенты', 'карьерный марафон', 'ярмарка вакан

В статье с большим количеством числительных они выделяются в качестве ключевых слов. Также выделяются именна собственные, тут, кажется, зависит от того, что мы хотим видеть в ключевых словах, в принципе, они могут быть полезными.Есть также проблема с переводом названий (наверняка то же случится с географическими названиями), например, 'career marathon' и 'карьерный марафон' на самом деле одно и то же! Кажется, что нужна какая-то база подобных соответствий. Некоторые выражения, являющиеся ключевыми словами, не распознаются, но слова, входящие в их состав, алгоритм определяет как ключевые. Как решить эту проблему мне пока непонятно, ведь установить распознавание ключевых выражений длиной 3, допустим, мы не можем, а определение коридора длины проблему не решит.
В целом, основная проблема заключается в преобработке текста, нужно усовершенствовать токенизацию, убирать цифры, расширить список стоп-слов.